In [ ]:
import os 
import importlib

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.Waveform import Waveform
from waffles.np04_data.ProtoDUNE_HD_APA_maps import APA_map # apa map
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid # channel grid

import integral_computation_function as integral_module
from integral_computation_function import channel_integral_computation

import waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots as utils_plots_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *


from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak import MyAnaPeak

import waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW as MyAnaPeak_NEW_module


In [ ]:
# Reading full streaming data

run_folder = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/full_streaming/no_selection"
fname = "processed_np04hd_raw_run027361_0001_dataflow0_datawriter_0_20240622T004415.hdf5_structured.hdf5"
filepath = os.path.join(run_folder, fname)
FS_wfset = load_structured_waveformset(filepath)  # load HDF5 structured waveform set  

In [ ]:
# Plotting original full-streaming waveforms --> Negative signals

importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, peak_bool = False, peak_beam_bool = False, inversion = False, input_signal_sign = "negative")

In [ ]:
# Peak finding analysis for beam event selection 
importlib.reload(MyAnaPeak_NEW_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW import MyAnaPeak_NEW

analysis_label = 'peak_finding'
analysis_class= MyAnaPeak_NEW

ip = IPDict(baseline_limits= [0, 300],
            n_std = 5,
            peak_distance= 500,
            beam_timeoffset_limits = [-40, -20]
            )

_ = FS_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf}), overwrite=True)

print('done\n\n')

In [ ]:
# Plotting full-streaming waveforms with peak finding results --> Negative signals

importlib.reload(utils_plots_module) 
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = False, peak_bool = False, peak_beam_bool = True, inversion = False, input_signal_sign = "negative", analysis_label = analysis_label)

In [ ]:
# Creo nuovo wfset per fare prove sulla selezione della finistra di full-streaming

# Lo slice adcs funziona per selezionare gli adcs, ma non trasla le altre informazioni 
# Tutte le info della wf non vengono modificare (timestamp, daq_window_timestamp, run_number,record_number,endpoint,channel,trigger_type)
# Viene solo modificato starting_tick che di default è 0 di solito
# Per quanto riguarda il wfset dopo che ho fatto apply, gli attributi rimango uguali compreso points_per_wf 187520


# Da fare: trovare 1/3 dell'ampiezza del picco e selezionare la finestra intorno 
# Da valutare: conviene rimuovere da prima la baseline? probabilmente si (magari usa metodo julio)

In [ ]:
prova_wfset = load_structured_waveformset(filepath)

In [ ]:
# Goal: cut full-streaming waveforms around the peak 
importlib.reload(MyAnaPeak_NEW_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW import MyAnaPeak_NEW
analysis_class = MyAnaPeak_NEW
analysis_label = 'finding_peaks'


_ = prova_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf}), overwrite=True)


In [ ]:
# To cut full-streaming waveforms around the peak

from waffles.data_classes.WaveformAdcs import WaveformAdcs
from waffles.Exceptions import GenerateExceptionMessage

def cut_full_streaming_window(
        waveform: WaveformAdcs,
        analysis_label: str
) -> None:
    """This method overwrites the adcs method of the given
    WaveformAdcs object.

    waveform: WaveformAdcs
        The waveform whose adcs will be modified
    analysis_label: str
    -....
        The baseline to subtract must be available 
        under the 'baseline' key of the result of the analysis
        whose label is given by this parameter, i.e. in
        waveform.analyses[analysis_label].result['baseline']
    """

    try:
        peak_index = waveform.analyses[analysis_label].result['beam_peak_index']
        peak_amplitude = waveform.analyses[analysis_label].result['beam_peak_amplitude']
        baseline = waveform.analyses[analysis_label].result['mean_baseline']
    except KeyError:
        raise Exception(
            GenerateExceptionMessage(
                1,
                "Problem finding 'beam_peak_index' or 'beam_peak_amplitude' "
            )
        )
    
    # bisogna trovare 1/3 dell'ampiezza
    clean_peak_amplitude = baseline - peak_amplitude

    #waveform._Waveform__slice_adcs(10000, 20000)

    return


In [ ]:

prova_wfset.apply(cut_full_streaming_window, analysis_label = analysis_label ,show_progress=False)

In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 20, show  = True, peak_bool = False, peak_beam_bool = False, inversion = False, input_signal_sign = "negative", analysis_label = analysis_label)

In [ ]:
# Altre cose utili di sotto

In [ ]:
print("\nFS_wfset waveformset:")
print("points_per_wf:", FS_wfset.points_per_wf)
print("runs:", FS_wfset.runs)
print("record_numbers:", FS_wfset.record_numbers)
print("available_channels:", FS_wfset.available_channels)
print("mean_adcs:", FS_wfset.mean_adcs)
print("mean_adcs_idcs:", FS_wfset.mean_adcs_idcs)

print("\nprova_wfset waveformset:")
print("points_per_wf:", prova_wfset.points_per_wf)
print("runs:", prova_wfset.runs)
print("record_numbers:", prova_wfset.record_numbers)
print("available_channels:", prova_wfset.available_channels)
print("mean_adcs:", prova_wfset.mean_adcs)
print("mean_adcs_idcs:", prova_wfset.mean_adcs_idcs)

print("\n --------------------------------------------- \n")
i=2
print("\nprova_wfset waveform:")
print("timestamp:", prova_wfset.waveforms[i].timestamp)
print("daq_window_timestamp:", prova_wfset.waveforms[i].daq_window_timestamp)
print("run_number:", prova_wfset.waveforms[i].run_number)
print("record_number:", prova_wfset.waveforms[i].record_number)
print("endpoint:", prova_wfset.waveforms[i].endpoint)
print("channel:", prova_wfset.waveforms[i].channel)
print("starting_tick:", prova_wfset.waveforms[i].starting_tick)
print("trigger_type:", prova_wfset.waveforms[i].trigger_type)

print("\n\FS_wfset waveform:")
print("timestamp:", FS_wfset.waveforms[i].timestamp)
print("daq_window_timestamp:", FS_wfset.waveforms[i].daq_window_timestamp)
print("run_number:", FS_wfset.waveforms[i].run_number)
print("record_number:", FS_wfset.waveforms[i].record_number)
print("endpoint:", FS_wfset.waveforms[i].endpoint)
print("channel:", FS_wfset.waveforms[i].channel)
print("starting_tick:", FS_wfset.waveforms[i].starting_tick)
print("trigger_type:", FS_wfset.waveforms[i].trigger_type)

In [ ]:
# Esempio funzioannete: To remove baseline

from waffles.data_classes.WaveformAdcs import WaveformAdcs
from waffles.Exceptions import GenerateExceptionMessage

def subtract_baseline(
        waveform: WaveformAdcs,
        baseline_analysis_label: str
) -> None:
    """This method overwrites the adcs method of the given
    WaveformAdcs object, by subtracting its baseline.

    waveform: WaveformAdcs
        The waveform whose adcs will be modified
    baseline_analysis_label: str
        The baseline to subtract must be available 
        under the 'baseline' key of the result of the analysis
        whose label is given by this parameter, i.e. in
        waveform.analyses[analysis_label].result['baseline']
    """

    try:
        baseline = waveform.analyses[baseline_analysis_label].result['baseline']

    except KeyError:
        raise Exception(
            GenerateExceptionMessage(
                1,
                "subtract_baseline()",
                f"The given waveform does not have the analysis"
                f" '{baseline_analysis_label}' in its analyses "
                "attribute, or it does, but the 'baseline' key "
                "is not present in its result."
            )
        )
    
    waveform._WaveformAdcs__set_adcs(
        waveform.adcs - baseline
    )

    return

ch_wfset.apply(subtract_baseline, baseline_analysis_label, show_progress=False)

In [ ]:
# single waveform plot

i=0

fig = go.Figure()

y = prova_wfset.waveforms[i].adcs 
x = np.arange(len(y))
    
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='lines',
    line=dict(width=0.5),
    showlegend=False))


fig.add_trace(go.Scatter(
    x=prova_wfset.waveforms[i].analyses[analysis_label].result['peak_time'],
    y=prova_wfset.waveforms[i].analyses[analysis_label].result['peak_amplitude'],
    mode='markers',
    marker=dict(color='red', size=8, symbol='circle'),
    name='Peaks'
    ))

fig.show()